In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import statsmodels.stats as STA

In [2]:
master = pd.DataFrame(pd.read_excel('QCLAB_AD_mice.xlsx'))
target = pd.DataFrame(pd.read_csv('studywide_stats_for_volume.txt', sep='\t'))

In [3]:
print(master.head())
print(target.head())

       Animal       Date Modality Genotype     DiseaseModel   Sex Treatment  \
0  190610-1:1 2019-08-01      MRI   APOE44  AD_genetic_risk  male      none   
1  190610_2:1 2019-08-01      MRI   APOE44  AD_genetic_risk  male      none   
2  190610_3:1 2019-08-01      MRI   APOE44  AD_genetic_risk  male      none   
3  190610-4:1 2019-08-01      MRI   APOE44  AD_genetic_risk  male      none   
4  190610-5:1 2019-08-01      MRI   APOE44  AD_genetic_risk  male      none   

  weight        DOB MaterialDecomposition  ... OffsetPerf1 Perf2_1p5  \
0   30.7 2018-04-03                   NaN  ...         NaN      45.0   
1   32.1 2018-04-03                   NaN  ...         NaN      59.0   
2   31.6 2018-04-03                   NaN  ...         NaN      78.0   
3   30.6 2018-04-03                   NaN  ...         NaN      96.0   
4   32.2 2018-04-03                   NaN  ...         NaN     115.0   

  OffsetPerf2 T1map1  OffsetT1map  T1map2  OffsetT2map  Effciency  LabelOptim  \
0         N

In [4]:
target.loc['sum'] = target.sum(axis = 0)
target.loc['sum'] = target.loc['sum']-target.iloc[0]

In [5]:
volume = target.drop(['ROI'], axis = 1)
volume = volume.div(pd.Series(volume.loc['sum'], index=volume.columns))
volume['ROI'] = target['ROI']
volume = volume.drop(['sum'])

In [6]:
mid = volume['ROI']
volume.drop(labels=['ROI'], axis=1,inplace = True)
volume.insert(0, 'ROI', mid)

In [7]:
header = volume.columns

In [8]:
volume.loc['type'] = 'N/A'
for i in range(len(header)):
    for j in range(len(master.loc[:]['SAMBA Brunno'])):
        if header[i] == master.loc[j]['SAMBA Brunno']:
            if master.loc[j]['Genotype'] == 'CVN':
                volume.loc['type'][i] = master.loc[j]['Treatment']
            else:
                volume.loc['type'][i] = master.loc[j]['Genotype']


In [9]:
volume.loc['encode'] = 'N/A'
encode = {
    'APOE22':2,
    'APOE33':3,
    'APOE44':4,
    'sedentary':5,
    'treadmill':6,
    'wheel_only':7
}
for k in range(len(header)):
    if volume.loc['type'][k] in encode.keys():
        volume.loc['encode'][k] = encode[volume.loc['type'][k]]


In [10]:
APOE = volume.copy(deep=True)
drop = []
for m in range(len(header)):
    if APOE.loc['encode'][m] not in [5,6,7]:
        drop.append(header[m])



In [11]:
for n,d in enumerate(drop):
    APOE.drop(d,axis=1,inplace=True)

In [12]:
APOE_comp = pd.DataFrame(columns=['ROI','P Value','R2'])
APOE_comp['ROI'] = volume['ROI']

In [13]:
APOE_comp.drop(APOE_comp.tail(2).index,inplace=True)

In [14]:
for l in range(APOE.shape[0]-2):
    x = np.array(APOE.loc['encode'],dtype=float)
    x = sm.add_constant(x)
    y = np.array(APOE.iloc[l][:],dtype=float)
    model = sm.OLS(y,x)
    result = model.fit()
    pvalues = result.pvalues
    r2 = result.rsquared
    APOE_comp.iloc[l][1] = pvalues[1]
    APOE_comp.iloc[l][2] = r2

In [15]:
rej, P_corr = STA.multitest.fdrcorrection(APOE_comp['P Value'])

In [16]:
APOE_comp['P corrected'] = P_corr

In [16]:
APOE_comp.to_csv('volume_APOE_comparison.csv')

In [17]:
APOE_sig = APOE_comp[(APOE_comp['P corrected'] < 0.01)]